# Land Cover Change - Deforestation and Crop Expansion
Change in land cover class (as defined by [Copernicus Global Land Cover](https://lcviewer.vito.be/about)): forest changing into cropland, and the reverse. Forest subtypes not disaggregated.  

Note that this is a parameterized widget; the specification passed to the API will not be renderable without the geostore identifier being inserted.

_Author: Peter Kerins_  
_Created: 15 Jun 2021_  
_Environment: jupyterlab_  

## Style
- Vega chart
- Ocean Watch country page - parameterized chart
- Time series
- Vertical bar
- Line for net change

## Data
Surface areas [calculated on Google Earth Engine](https://code.earthengine.google.com/1a106a75689826e6bf7283b7d7c7f6f5); outputs combined and restructured via [Python script](../../datasets/ocn_calcs_016_land_cover_change/land-cover-change-structuring.py).  
Carto: [soc_104b](https://resourcewatch.carto.com/u/wri-rw/dataset/soc_104b_land_cover_change)  

## Preparation

In [2]:
import json
from vega import Vega
from IPython.display import display

In [3]:
def Vega(spec):
    bundle = {}
    bundle['application/vnd.vega.v5+json'] = spec
    display(bundle, raw=True)

In [4]:
widget_width = 400
widget_height = 300

# Widget  

## Demo Query
`gid_0 = <territory>` used as stand-in for parameterization
```sql
SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit 
FROM soc_104b_land_cover_change data INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0 
WHERE variable IN ('cropForest', 'forestCrop') AND gadm.gid_0 ILIKE 'BRA' 
ORDER BY gadm.gid_0 ASC, date ASC, variable DESC
```

Second query for net forest gain/loss (simply wraps prior query)
```sql
SELECT gid_0, date, SUM(value) AS net_value 
    FROM 
        (SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit 
             FROM soc_104b_land_cover_change data
        INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0
        WHERE variable IN ('cropForest', 'forestCrop') AND gadm.gid_0 ILIKE 'BRA' ORDER BY gadm.gid_0 ASC, date ASC, variable DESC )
        AS query_inner 
 GROUP BY gid_0, date ORDER BY gid_0 ASC
```

## Parameterized Query
```sql
SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit 
FROM soc_104b_land_cover_change data INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0 
WHERE variable IN ('cropForest', 'forestCrop') AND gadm.{{geostore_env}} ILIKE '{{geostore_id}}' 
ORDER BY gadm.gid_0 ASC, date ASC, variable DESC
```

Second query for net forest gain/loss (simply wraps prior query)
```sql
SELECT gid_0, date, SUM(value) AS net_value 
    FROM 
        (SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit 
             FROM soc_104b_land_cover_change data
        INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0
        WHERE variable IN ('cropForest', 'forestCrop') AND gadm.{{geostore_env}} ILIKE '{{geostore_id}}' ORDER BY gadm.gid_0 ASC, date ASC, variable DESC )
        AS query_inner 
 GROUP BY gid_0, date ORDER BY gid_0 ASC
```

## Specification
Code presented in notebook is already written for RW (rather than standard Vega). This does not extent to parameterization.

In [5]:
spec=json.loads("""
{
  "autosize": {"type": "pad", "resize": true, "contains": "padding"},
  "signals": [
    {
      "name": "hover",
      "value": null,
      "on": [
        {"events": "@cell:mouseover", "update": "datum"},
        {"events": "@cell:mouseout", "update": "null"}
      ]
    }
  ],
  "data": [
    {
      "name": "table",
      "url": "https://wri-rw.carto.com/api/v2/sql?q= SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit FROM soc_104b_land_cover_change data INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0 WHERE variable IN ('cropForest', 'forestCrop') AND gadm.{{geostore_env}} ILIKE '{{geostore_id}}' ORDER BY gadm.gid_0 ASC, date ASC, variable DESC",
      "format": {
        "type": "json", "property": "rows"
      },
      "transform": [
        {
          "type": "formula",
          "expr": "abs(datum.value)",
          "as": "abs_value"
        },
        {
          "type": "formula",
          "expr": "(datum.variable=='cropForest' ? 'Cropland to forest' : 'Forest to cropland')",
          "as": "alias"
        }
      ]
    },
    {
      "name": "net",
      "url": "https://wri-rw.carto.com/api/v2/sql?q=SELECT gid_0, date, SUM(value) AS net_value FROM (SELECT gadm.gid_0, gadm.name_0, variable, date, CASE WHEN variable ILIKE 'cropForest' THEN value ELSE value * -1 END AS value, unit FROM soc_104b_land_cover_change data INNER JOIN gadm36_0 gadm ON gadm.gid_0 = data.gid_0 WHERE variable IN ('cropForest', 'forestCrop') AND gadm.{{geostore_env}} ILIKE '{{geostore_id}}' ORDER BY gadm.gid_0 ASC, date ASC, variable DESC ) AS query_inner GROUP BY gid_0, date ORDER BY gid_0 ASC",
      "format": {
        "type": "json", "property": "rows"
      }
    }
  ],
  "scales": [
    {
      "name": "xscale",
      "type": "band",
      "domain": {"data": "table", "field": "date"},
      "range": "width",
      "paddingInner": 0.2,
      "paddingOuter": 0.2
    },
    {
      "name": "yscale",
      "type": "linear",
      "domain": {"data": "table", "field": "value"},
      "range": "height",
      "round": true,
      "zero": true,
      "nice": true
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": [
        "Cropland to forest",
        "Forest to cropland"
      ],
      "range": [
        "#0F870F",
        "#965596"
      ]
    },
    {
      "name": "line-color",
      "type": "ordinal",
      "domain": [
        "Net forest gain"
      ],
      "range": [
        "black"
      ]
    }
  ],
  "axes": [
    {
      "orient": "left",
      "scale": "yscale",
      "title": "Area (sq. km)",
      "titleFont": "Lato",
      "encode": {
        "labels": {
          "interactive": false,
          "enter": {
          },
          "update": {
            "align": {"value": "right"},
            "baseline": {"value": "middle"},
            "dx": {"value": 4},
            "fill": {"value": "#717171"}
          },
          "hover": {
            "fill": {"value": "firebrick"}
          }
        }
      }
    },
    {
      "orient": "bottom",
      "scale": "xscale",
      "tickSize": 0,
      "labelPadding": 4,
      "zindex": 1,
      "encode": {
        "labels": {
          "interactive": false,
          "enter": {
          },
          "update": {
            "fill": {"value": "#717171"}
          },
          "hover": {
            "fill": {
              "value": "black"
            }
          }
        }
      }
    }
  ],
  "marks": [
    {
      "name": "bars",
      "from": {"data": "table"},
      "type": "rect",
      "encode": {
        "enter": {
          "x": {"scale": "xscale", "field": "date"},
          "width": {"scale": "xscale", "band": 1.0},
          "y": {"scale": "yscale", "value": 0},
          "y2": {"scale": "yscale", "field": "value"},
          "fill": {"scale": "color", "field": "alias"}
        },
        "hover": {"fillOpacity": {"value": 0.5}},
        "update": {"fillOpacity": {"value": 1}}
      }
    },
    {
      "name": "line",
      "from": {"data": "net"},
      "type": "line",
      "encode": {
        "enter": {
          "x": {
            "signal": "scale('xscale',datum.date)+bandwidth('xscale')/2"
          },
          "y": {"scale": "yscale", "field": "net_value"},
          "stroke": {"value": "black"}
        }
      }
    }
  ],
  "legends": [
    {
      "fill": "color",
      "orient": "right",
      "labelFont": "Lato"
    },
    {
      "fill": "line-color",
      "orient": "right",
      "labelFont": "Lato",
      "encode": {
        "symbols": {
          "update": {
            "size": {"value": 80},
            "stroke": {"value": "black"},
            "shape": {
              "value": "stroke"
            }
          }
        }
      }
    }
  ],
  "interaction_config": [
    {
      "name": "tooltip",
      "config": {
        "fields": [
          {
            "column": "abs_value",
            "property": "Area (sq. km)",
            "type": "number",
            "format": ",.1f"
          }
        ]
      }
    }
  ]
}
""")


In [6]:
vega_view=dict(spec)

## DEFINE SIZE
vega_view['height'] = widget_height
vega_view['width'] = widget_width

## PARAMETERIZE WIDGET
vega_view['data'][0]['url']= vega_view['data'][0]['url'].replace('{{geostore_env}}','geostore_prod')
vega_view['data'][0]['url'] = vega_view['data'][0]['url'].replace('{{geostore_id}}','804b97bc661ae04c2b899d71ecaeb858')
vega_view['data'][1]['url']= vega_view['data'][1]['url'].replace('{{geostore_env}}','geostore_prod')
vega_view['data'][1]['url'] = vega_view['data'][1]['url'].replace('{{geostore_id}}','804b97bc661ae04c2b899d71ecaeb858')

## ADJUST X AXIS
vega_view['axes'][0]['labelBaseline']= "top"

## FONT
vega_view['axes'][1]['titleFont'] = 'Arial'
vega_view['legends'][0]['labelFont'] = 'Arial'
vega_view['legends'][1]['labelFont'] = 'Arial'


Vega(vega_view)



# RW API
Parent dataset [soc.104.rw0 Global Land Cover](https://resourcewatch.org/data/explore/Global-Land-Cover-UN-FAO-LCCS-Classification) ([Admin](https://resourcewatch.org/admin/data/datasets/b2f00f99-46ed-43e6-a7a1-a5809d9369d4/edit))    
[API ID](https://api.resourcewatch.org/v1/dataset/b2f00f99-46ed-43e6-a7a1-a5809d9369d4/) `b2f00f99-46ed-43e6-a7a1-a5809d9369d4`  
[Widgets](https://api.resourcewatch.org/v1/dataset/b2f00f99-46ed-43e6-a7a1-a5809d9369d4/widget)

Widgets created via [widget scripts](https://github.com/resource-watch/data-team-tools/tree/master/advanced_widget_writer).  

## Demo: Brazil

In [2]:
dataset_id = 'b2f00f99-46ed-43e6-a7a1-a5809d9369d4'
name = 'Land Cover Change in Brazil, 2015–2019: Deforestation and Crop Expansion'
description = 'Area of land that changed from forest to cropland or vice versa, year on year'

- [Brazil](https://api.resourcewatch.org/v1/widget/0e8df4ea-d191-4c46-a626-268a4f23d18a?env=production) `0e8df4ea-d191-4c46-a626-268a4f23d18a`  

## Parameterized Version

In [6]:
dataset_id = 'b2f00f99-46ed-43e6-a7a1-a5809d9369d4'
name = 'Deforestation and Crop Expansion, 2015–2019'
description = 'Area of land that changed from forest to cropland or vice versa, year on year'

- [Country Page](https://api.resourcewatch.org/v1/widget/15b5ec89-0ea6-4cd4-beac-7caceb80e42c?env=production) `15b5ec89-0ea6-4cd4-beac-7caceb80e42c`  